In [1]:
import pandas as pd

In [2]:
def preprocess(name, csvfile):
    df = pd.read_csv(csvfile, index_col=0)
    tfrmt = lambda x: x if pd.isna(x) else f"{x / 3600:02.0f}:{x / 60 % 60:02.0f}:{x % 60:02.0f}"
    df['duration'] = df['time'].apply(tfrmt)
    df.columns = [f"{name}_{x}" for x in df.columns]
    return df

In [3]:
pyro_mixed = preprocess("pyro_mixed", "201104_1633_pyro_mixed.csv")
numpyro_mixed = preprocess("numpyro_mixed", "201113_0434_numpyro_mixed.csv")
numpyro_comprehensive = preprocess("numpyro_comprehensive", "201113_0515_numpyro_comprehensive.csv")
numpyro_generative = preprocess("numpyro_generative", "201113_0546_numpyro_generative.csv")

In [4]:
results = pd.concat([pyro_mixed, numpyro_mixed, numpyro_comprehensive, numpyro_generative], axis=1)
results['example'] = results.index.map(lambda x: x.split("-")[1])
results['data'] = results.index.map(lambda x: x.split("-")[0])
results = results.sort_values(by='example')

In [5]:
summary = results[["example", "data", "pyro_mixed_status", "numpyro_mixed_status", "numpyro_comprehensive_status", "numpyro_generative_status"]]
print(summary.to_markdown(index=False))

| example                   | data                 | pyro_mixed_status   | numpyro_mixed_status   | numpyro_comprehensive_status   | numpyro_generative_status   |
|:--------------------------|:---------------------|:--------------------|:-----------------------|:-------------------------------|:----------------------------|
| accel_gp                  | mcycle_gp            | error               | error                  | error                          | error                       |
| arK                       | arK                  | success             | mismatch               | mismatch                       | mismatch                    |
| arma11                    | arma                 | success             | error                  | error                          | error                       |
| blr                       | sblrc                | success             | success                | success                        | error                       |
| blr                 

In [6]:
tex_summary = summary.replace("success", "smark").replace("mismatch", "mmark").replace("error", "emark")
print(tex_summary.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
                   example &                  data & pyro\_mixed\_status & numpyro\_mixed\_status & numpyro\_comprehensive\_status & numpyro\_generative\_status \\
\midrule
                  accel\_gp &             mcycle\_gp &             emark &                emark &                        emark &                     emark \\
                       arK &                   arK &             smark &                mmark &                        mmark &                     mmark \\
                    arma11 &                  arma &             smark &                emark &                        emark &                     emark \\
                       blr &                 sblrc &             smark &                smark &                        smark &                     emark \\
                       blr &                 sblri &             smark &                smark &                        smark &                     emark \\
            